In [ ]:
%run package_import.ipynb

In [ ]:
df_listings = pd.read_sql_query(

'''
select 
    name,     
    neighbourhood_cleansed,
    property_type,  
    room_type,  
    accommodates,
    bathrooms,  
    beds,
    bed_type,
    amenities,  
    cleaning_fee, 
    b.month,
    b.avg_price,
    b.min_price,
    b.max_price
from raw_listings a
inner join (
    select  listing_id,  
        month(date) as month,
        avg(price) as avg_price,  
        min(price) as min_price,  
        max(price) as max_price
    from raw_calendar 
    where price is not null 
        or price != 0
    group by listing_id, month(date)
    ) b
on a.id = b.listing_id
'''
,sqlEngine)

In [ ]:
# read listing
df_listings = pd.read_sql_query(

'''
select 
    id,
    name,     
    neighbourhood_cleansed,
    property_type,  
    room_type,  
    accommodates,
    bathrooms,  
    beds,
    bed_type,
    amenities,  
    cleaning_fee
from raw_listings a
'''
,sqlEngine)

#read calendar
df_calendar = pd.read_sql_query(

'''
select *,  
    month(date) as month
from raw_calendar 
where price != 0
'''
,sqlEngine)

df_calendar_month = df_calendar.groupby(
    ['listing_id', 'month'],
    as_index=False
).agg(
    median_price=pd.NamedAgg(column="price", aggfunc="median"),
    avg_price=pd.NamedAgg(column="price", aggfunc="mean"),  
    min_price=pd.NamedAgg(column="price", aggfunc="min"),  
    max_price=pd.NamedAgg(column="price", aggfunc="max")
)

df_listings = df_listings.merge(
    df_calendar_month,  
    left_on='id',  
    right_on='listing_id',
    how='inner'
)

In [ ]:
def convert_amenities_to_columns(df: pd.DataFrame) -> pd.DataFrame:
    '''  
    input:
    df: dataframe whose amenities columns to be created

    output:
    return dataframe with amenities columns
    
    '''


    # remove { and } from each row in dataframe
    replacement = {'{': '', 
                    '}': '',
                    '"': '',
                    ' ': '_',
                    '/': '_'
                    }

    df = df.replace({'amenities':replacement}, regex=True)
    # lower case
    df['amenities'] = df['amenities'].str.lower()
    # convert amenities to a list
    amenities_list = pd.unique(df['amenities'])

    # read unique items in amenities
    amenities_column = []
    for amenity in amenities_list:
        for item in amenity.split(','):
            # if item is blank and is not in the amenities_column
            # then append it
            # otherwise skip it
            if item != '' and item not in amenities_column:
                amenities_column.append(item) 

    # if the column name exist in the column "amenities"
    # flag 1; otherwise flag 0
    for col in amenities_column:
        df[col] = df['amenities'].apply(
            lambda x: 1 if col in x else 0
        )

    return df

In [ ]:
df_listings = convert_amenities_to_columns(df_listings)

In [ ]:
df_listings['median_price'].hist()

In [ ]:
df_listings.sort_values(by='median_price', ascending=False).head()

In [ ]:
df_listings.groupby(['neighbourhood_cleansed','month'], as_index=False).agg(
    median_price=pd.NamedAgg(column='median_price', aggfunc='median'),
    avg_price=pd.NamedAgg(column='avg_price', aggfunc='mean')
).sort_values(by='median_price', ascending=False)

In [ ]:
# need to convert amenities to columns
# need to classify price into bin and then do analysis